In [1]:
import numpy as np
import pandas as pd
import onnxruntime as rt

data = pd.read_csv("datasets/diabetes_risk_dataset.csv")

numerical_cols = [
    'age', 'bmi', 'blood_pressure', 'fasting_glucose_level',
    'insulin_level', 'HbA1c_level', 'cholesterol_level',
    'triglycerides_level', 'daily_calorie_intake',
    'sugar_intake_grams_per_day', 'sleep_hours',
    'stress_level', 'waist_circumference_cm'
]

categorical_cols = [
    'gender', 'physical_activity_level', 'family_history_diabetes'
]

feature_cols = numerical_cols + categorical_cols

sample = data.loc[[0], feature_cols].copy()


sess = rt.InferenceSession("diabetes_risk_model.onnx")

input_dict = {}

for col in numerical_cols:
    input_dict[col] = sample[col].astype(np.float32).values.reshape(-1, 1)

for col in categorical_cols:
    input_dict[col] = sample[col].astype(str).values.reshape(-1, 1)

logit_pred = sess.run(None, input_dict)[0]

risk_scaled = 1 / (1 + np.exp(-logit_pred))
risk_final = risk_scaled * 100

print("Predicción final (ONNX):", float(risk_final))

print("Valor real:", float(data.loc[0, 'diabetes_risk_score']))

Predicción final (ONNX): 50.332313537597656
Valor real: 52.3


/var/folders/hw/5mshbnp16hqd906xl4fc25qr0000gn/T/ipykernel_14643/3669901664.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Predicción final (ONNX):", float(risk_final))
